# 텐서플로를 이용하여 단순 RNN 모델 만들기

### 10.12 단순 RNN과 LSTM, GRU 모델의 비교 (1) - 시퀀스 데이터 준비

In [14]:
import numpy as np

# 0.0, 0.1, .. 증가하는 시퀀스 데이터를 생성함
# seq_len 길이를 가지는 시퀀스 데이터를 size 갯수만큼 생성한다
def sequence_gen(size, seq_len):
    # 비어있는 넘파이 배열을 생성한다.
    seq_X = np.empty(shape=(size, seq_len, 1))
    Y = np.empty(shape=(size,))

    for i in range(size):
        # [0, 0.1, 0.2, .. ]같은 시퀀스와 Y 값을 size 갯수만큼 생성
        c = np.linspace(i/10, (i+seq_len-1)/10, seq_len)
        seq_X[i] = c[:, np.newaxis]   # 새로운 축을 하나 더 추가함
        Y[i] = (i+seq_len) / 10       # 목표값을 생성함

    return seq_X, Y

# 길이가 16인 시퀀스 8개를 훈련용으로 만든다
n, seq_len = 8, 16   
train_seq_X, train_Y = sequence_gen(n, seq_len)

In [15]:
print('훈련용 데이터')
for i in range(n):
    print(train_seq_X[i].flatten(), train_Y[i])

half_n, offset = int(n/2), 1.0
test_seq_X = train_seq_X[:half_n] + offset
test_Y = train_Y[:half_n] + offset

print('검증용 데이터')
for i in range(half_n):
    print(test_seq_X[i].flatten(), test_Y[i])

훈련용 데이터
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5] 1.6
[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6] 1.7
[0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7] 1.8
[0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8] 1.9
[0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9] 2.0
[0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2. ] 2.1
[0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1] 2.2
[0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2] 2.3
검증용 데이터
[1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5] 2.6
[1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6] 2.7
[1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6 2.7] 2.8
[1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6 2.7 2.8] 2.9


### 10.13 단순 RNN과 LSTM, GRU 모델의 비교 (1) - 성능 비교

In [16]:
import tensorflow as tf
n_units = 256
simpleRNN_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(units = n_units, return_sequences=False,
                              input_shape=[seq_len, 1]),
    tf.keras.layers.Dense(1)
])

simpleRNN_model.compile(optimizer ='adam', loss = 'mse')
simpleRNN_model.fit(train_seq_X, train_Y, epochs=100)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 5.6031
Epoch 2/100
1/1 [==============================] - 0s 12ms/step - loss: 0.5418
Epoch 3/100
1/1 [==============================] - 0s 15ms/step - loss: 0.1615
Epoch 4/100
1/1 [==============================] - 0s 12ms/step - loss: 0.5217
Epoch 5/100
1/1 [==============================] - 0s 15ms/step - loss: 0.1708
Epoch 6/100
1/1 [==============================] - 0s 14ms/step - loss: 0.0369
Epoch 7/100
1/1 [==============================] - 0s 18ms/step - loss: 0.2574
Epoch 8/100
1/1 [==============================] - 0s 14ms/step - loss: 0.1924
Epoch 9/100
1/1 [==============================] - 0s 15ms/step - loss: 0.0365
Epoch 10/100
1/1 [==============================] - 0s 14ms/step - loss: 0.0091
Epoch 11/100
1/1 [==============================] - 0s 14ms/step - loss: 0.0749
Epoch 12/100
1/1 [==============================] - 0s 14ms/step - loss: 0.1055
Epoch 13/100
1/1 [==============================] -

In [18]:
result = simpleRNN_model.predict(test_seq_X)
result = result.flatten()
print('예측값 :', result)
print('실제값 :', test_Y)

예측값 : [2.4342813 2.4811897 2.5235074 2.5617518]
실제값 : [2.6 2.7 2.8 2.9]


In [19]:
LSTM_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units = n_units, return_sequences=False,
                         input_shape=[seq_len, 1]),
    tf.keras.layers.Dense(1)
])

LSTM_model.compile(optimizer ='adam', loss = 'mse')
LSTM_model.fit(train_seq_X, train_Y, epochs=100)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 3.6272
Epoch 2/100
1/1 [==============================] - 0s 41ms/step - loss: 2.5411
Epoch 3/100
1/1 [==============================] - 0s 41ms/step - loss: 1.5906
Epoch 4/100
1/1 [==============================] - 0s 39ms/step - loss: 0.7648
Epoch 5/100
1/1 [==============================] - 0s 41ms/step - loss: 0.1571
Epoch 6/100
1/1 [==============================] - 0s 36ms/step - loss: 0.1079
Epoch 7/100
1/1 [==============================] - 0s 52ms/step - loss: 0.5497
Epoch 8/100
1/1 [==============================] - 0s 41ms/step - loss: 0.3887
Epoch 9/100
1/1 [==============================] - 0s 39ms/step - loss: 0.1108
Epoch 10/100
1/1 [==============================] - 0s 36ms/step - loss: 0.0109
Epoch 11/100
1/1 [==============================] - 0s 37ms/step - loss: 0.0453
Epoch 12/100
1/1 [==============================] - 0s 45ms/step - loss: 0.1184
Epoch 13/100
1/1 [==============================] -

In [20]:
result = LSTM_model.predict(test_seq_X)
result = result.flatten()
print('예측값 :', result)
print('실제값 :', test_Y)

예측값 : [2.5833712 2.6695101 2.752849  2.8334446]
실제값 : [2.6 2.7 2.8 2.9]


In [21]:
GRU_model = tf.keras.Sequential([
    tf.keras.layers.GRU(units = n_units, return_sequences=False, 
                        input_shape=[seq_len, 1]),
    tf.keras.layers.Dense(1)
])

GRU_model.compile(optimizer ='adam', loss = 'mse')
GRU_model.fit(train_seq_X, train_Y, epochs=100)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 3.8772
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 2.8685
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 2.0021
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2664
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 0.6657
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 0.2264
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 0.0097
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 0.1013
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 0.3522
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 0.3954
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - loss: 0.2687
Epoch 12/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1251
Epoch 13/100
1/1 [==============================] -

In [22]:
result = GRU_model.predict(test_seq_X)
result = result.flatten()
print('예측값 :', result)
print('실제값 :', test_Y)

예측값 : [2.5794032 2.672765  2.7653737 2.8572187]
실제값 : [2.6 2.7 2.8 2.9]
